In [ ]:
import zipfile
import random
import tensorflow as tf
from tensorflow import keras
import h5py
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
import scipy
import matplotlib.cm as cm
%matplotlib inline
import json
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import glob
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import imageio

In [ ]:
# Setting the paths to train dataset folders

#train_folder = '/kaggle/input/acrima-dataset/train'
glaucoma_folder = os.path.join('/kaggle/input/acrima-dataset/train/Glaucoma')
non_glaucoma_folder = os.path.join('/kaggle/input/acrima-dataset/train/Non Glaucoma')

In [ ]:
# Loading the glaucoma images

glaucoma_images = []
for image_file in os.listdir(glaucoma_folder):
    image_path = os.path.join(glaucoma_folder, image_file)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
    resized_image = cv2.resize(image_rgb, (224, 224))
    glaucoma_images.append(resized_image)

In [ ]:
# Load the non-glaucoma images

non_glaucoma_images = []
for image_file in os.listdir(non_glaucoma_folder):
    image_path = os.path.join(non_glaucoma_folder, image_file)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
    resized_image = cv2.resize(image_rgb, (224, 224))
    non_glaucoma_images.append(resized_image)

In [ ]:
# Converting the image data to numpy arrays

glaucoma_images = np.array(glaucoma_images)
non_glaucoma_images = np.array(non_glaucoma_images)

In [ ]:
print(glaucoma_images.shape)
print(non_glaucoma_images.shape)
print(glaucoma_images[0].shape)
print(non_glaucoma_images[0].shape)

In [ ]:
# Creating labels for the images (1 for glaucoma, 0 for non-glaucoma)

glaucoma_labels = np.ones(len(glaucoma_images))
non_glaucoma_labels = np.zeros(len(non_glaucoma_images))

In [ ]:
gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, 
    height_shift_range=0.1, shear_range=0.15, zoom_range=0.1, 
    channel_shift_range=10., horizontal_flip=True)

In [ ]:
#Augmentation for glaucoma images
aug_images = []
for i in range (glaucoma_images.shape[0]):
    image = glaucoma_images[i]
    image = np.expand_dims(image,0)
    aug_iter = gen.flow(image)
    for j in range(3):
        augmented_image = next(aug_iter)[0].astype(np.uint8)
        aug_images.append(augmented_image)

In [ ]:
aug_images = np.array(aug_images)
aug_images.shape
#plt.imshow(aug_images[0]) #works
augmented_glaucoma_images = np.concatenate((glaucoma_images, aug_images), axis=0)
augmented_glaucoma_images = augmented_glaucoma_images[:1000]
augmented_glaucoma_images.shape

In [ ]:
#Augmentation for non glaucoma images
aug_images = []
for i in range (non_glaucoma_images.shape[0]):
    image = non_glaucoma_images[i]
    image = np.expand_dims(image,0)
    aug_iter = gen.flow(image)
    for j in range(4):
        augmented_image = next(aug_iter)[0].astype(np.uint8)
        aug_images.append(augmented_image)

In [ ]:
aug_images = np.array(aug_images)
aug_images.shape
#plt.imshow(aug_images[0]) #works
augmented_non_glaucoma_images = np.concatenate((non_glaucoma_images, aug_images), axis=0)
augmented_non_glaucoma_images = augmented_non_glaucoma_images[:1000]
augmented_non_glaucoma_images.shape

In [ ]:
# Create the corresponding labels for the augmented data

num_samples = 1000
augmented_glaucoma_labels = np.ones((num_samples,))
augmented_non_glaucoma_labels = np.zeros((num_samples,))
print(augmented_glaucoma_labels.shape)
print(augmented_non_glaucoma_labels.shape)

In [ ]:
# Concatenate the augmented data

X_train = np.concatenate((augmented_glaucoma_images, augmented_non_glaucoma_images), axis=0)
y_train = np.concatenate((augmented_glaucoma_labels, augmented_non_glaucoma_labels), axis=0)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
# Generate random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)

# Shuffling the indices
indices = np.arange(len(X_train))
np.random.shuffle(indices)

# Reordering X_train and y_train using the shuffled indices
X_train_shuffled = X_train[indices]
y_train_shuffled = y_train[indices]


In [ ]:
print(X_train_shuffled.shape)
print(y_train_shuffled.shape)
X_train = X_train_shuffled
y_train = y_train_shuffled

In [ ]:
fig=plt.figure(figsize=(10,10))
columns = 4
rows = 3
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(X_train[i])
    if y_train[i] == 1:
        plt.title('advance_glaucoma')
    else:
        plt.title('normal_control')
    
plt.show()

In [ ]:
from keras import backend as K

IMG_CHANNELS=3
IMG_WIDTH = 224
IMG_HEIGHT = 224

if K.image_data_format() == 'channels_first':
    input_shape = (IMG_CHANNELS, IMG_WIDTH, IMG_HEIGHT)
else:
    input_shape = (IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS)

In [ ]:
# Setting the paths to test dataset folders

#train_folder = '/kaggle/input/acrima-dataset/train'
glaucoma_folder = os.path.join('/kaggle/input/acrima-dataset/test/Glaucoma')
non_glaucoma_folder = os.path.join('/kaggle/input/acrima-dataset/test/Non Glaucoma')


# Loading the glaucoma images
glaucoma_images = []
for image_file in os.listdir(glaucoma_folder):
    image_path = os.path.join(glaucoma_folder, image_file)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
    resized_image = cv2.resize(image_rgb, (224, 224))
    glaucoma_images.append(resized_image)
    

# Load the non-glaucoma images

non_glaucoma_images = []
for image_file in os.listdir(non_glaucoma_folder):
    image_path = os.path.join(non_glaucoma_folder, image_file)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB format
    resized_image = cv2.resize(image_rgb, (224, 224))
    non_glaucoma_images.append(resized_image)
    
    
# Converting the image data to numpy arrays

glaucoma_images = np.array(glaucoma_images)
non_glaucoma_images = np.array(non_glaucoma_images)

print(glaucoma_images.shape)
print(non_glaucoma_images.shape)
print(glaucoma_images[0].shape)
print(non_glaucoma_images[0].shape)


# Creating labels for the images (1 for glaucoma, 0 for non-glaucoma)

glaucoma_labels = np.ones(len(glaucoma_images))
non_glaucoma_labels = np.zeros(len(non_glaucoma_images))

In [ ]:
X_test = np.concatenate((glaucoma_images, non_glaucoma_images), axis=0)
y_test = np.concatenate((glaucoma_labels, non_glaucoma_labels), axis=0)
print(X_test.shape)
print(y_test.shape)
print(y_test)

In [ ]:
def create_model(input_shape, with_summary):
    wd = 0.0001
    model = Sequential()
    model.add(Conv2D(64, kernel_size=5, padding="same", input_shape=input_shape, activation = 'relu', kernel_regularizer=regularizers.l2(wd)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(64, kernel_size=3, padding="same", activation = 'relu', kernel_regularizer=regularizers.l2(wd)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Conv2D(64, kernel_size=3, padding="same", activation = 'relu', kernel_regularizer=regularizers.l2(wd)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())

    model.add(Conv2D(64, kernel_size=3, padding="same", activation = 'relu', kernel_regularizer=regularizers.l2(wd)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    #model.add(Conv2D(32, kernel_size=3, padding="same", activation = 'relu', kernel_regularizer=regularizers.l2(wd)))
    #model.add(BatchNormalization())
   #model.add(Dropout(0.1))
    
    #model.add(Conv2D(128, kernel_size=3, padding="valid", activation = 'relu',kernel_regularizer=regularizers.l2(wd)))
    #model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    #model.add(BatchNormalization())
    #model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(units=1000, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(units=10, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Dense(units=5, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Dense(1))
    model.add(Activation("sigmoid"))


    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    if with_summary:
        model.summary()

    return model
  


In [ ]:
def lr_schedule(epoch):
  lrate = 0.001
  if epoch > 75:
    lrate = 0.0001
  if epoch > 100:
    lrate = 0.001
  return lrate

In [ ]:
# Train the CNN Model
from keras import regularizers
from keras.callbacks import LearningRateScheduler
input_shape = (224, 224, 3)
model = create_model(input_shape=input_shape, with_summary=True)
history = model.fit(X_train, y_train,batch_size=64,epochs=100,validation_data=(X_test,y_test), callbacks=[LearningRateScheduler(lr_schedule)],verbose=1)
print(model.evaluate(X_test, y_test))

In [ ]:
from tensorflow.keras.models import load_model
model.save('ACRIMA_rgb_99.29.h5')

In [ ]:
#Saving - json file & .h5
model_json = model.to_json()
with open('model.json','w') as json_file:
  json_file.write(model_json)
model.save_weights('ACRIMA_rgb_99.29_weights.h5')

In [ ]:
print(history.history.keys())
#  "Accuracy"
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='lower right')
plt.show()
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Detrimine the predictions 
import numpy as np
predictions = model.predict(X_test)
predictions=np.array(predictions)
print(predictions.shape)
print(predictions)

In [ ]:
model.test_on_batch(X_test, y_test)
model.metrics_names

In [ ]:
history 

In [ ]:
history.history

In [ ]:
Y_prob = model.predict(X_test)
print(Y_prob)

In [ ]:
Y_pred = np.round(Y_prob)
print(Y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
print('The accuracy obtained on the test set is:', accuracy_score(Y_pred,y_test))

In [ ]:
accuracy_score(y_test, Y_pred, )

In [ ]:
import seaborn as sns
def draw_confusion_matrix(true,preds):
  conf_matx = confusion_matrix(true, preds)
  sns.heatmap(conf_matx, annot=True,annot_kws={"size": 12},fmt='g', cbar=False, cmap="viridis")
  plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, Y_pred))

In [ ]:
from sklearn.metrics import classification_report 
draw_confusion_matrix(y_test,Y_pred)
print('Confusion Matrix')
target_names = ['Advance Glaucoma', 'Normal Control']
print(classification_report(y_test, Y_pred, target_names=target_names))

In [ ]:
# Setup
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc

# Get ROC curve FPR and TPR from true labels vs score values
fpr, tpr, _ = roc_curve(y_test, Y_pred)
# Calculate ROC Area Under the Curve (AUC) from FPR and TPR data points
roc_auc = auc(fpr, tpr)
# Calculate precision and recall from true labels vs score values
precision, recall, _ = precision_recall_curve(y_test, Y_pred)
plt.figure(figsize=(8, 3))
plt.subplot(1,1,1)
lw = 2
plt.plot(fpr, tpr, color='red', lw=lw, label='ROC curve (area = %0.4f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.grid(True)

In [ ]:
fig = plt.figure(figsize=(10, 10))
columns = 3
rows = 2
random_number = np.random.randint(0, X_test.shape[0] - 26)

for i in range(1, columns * rows + 1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(X_test[i + random_number])
    gt = ['normal_control', 'advance_glaucoma']
    prediction_index = int(i + random_number)
    plt.title('Infection likelihood {:.1%}\n Ground Truth: {}'.format(float(predictions[prediction_index]), gt[int(y_test[prediction_index])]))

plt.show()
